# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 14 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub profile', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn profile',
   'u

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
3 days ago
•
530k
•
878
deepseek-ai/DeepSeek-OCR
Updated
7 days ago
•
1.66M
•
2.31k
moonshotai/Kimi-Linear-48B-A3B-Instruct
Updated
32 minutes ago
•
7.91k
•
257
meituan-longcat/LongCat-Video
Updated
3 days ago
•
916
•
234
briaai/FIBO
Updated
2 days ago
•
1.79k
•
148
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
724
724
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
15.6k
15.6k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
on
Zero
365
365
DeepSeek OCR Demo
🆘

In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\n3 days ago\n•\n530k\n•\n878\ndeepseek-ai/DeepSeek-OCR\nUpdated\n7 days ago\n•\n1.66M\n•\n2.31k\nmoonshotai/Kimi-Linear-48B-A3B-Instruct\nUpdated\nabout 1 hour ago\n•\n7.91k\n•\n257\nmeituan-longcat/LongCat-Video\nUpdated\n3 days ago\n•\n916\n•\n234\nbriaai/FIBO\nUpdated\n2 days ago\n•\n1.79k\n•\n149\nBrowse 1M+ models\nSpaces\nRunning\non\nCPU Upgrade\n724\n72

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the vibrant AI community building the future of machine learning. Serving as the central hub for machine learning engineers, scientists, and enthusiasts around the world, Hugging Face offers a collaborative platform where users can create, share, and discover state-of-the-art models, datasets, and applications.

The platform hosts over **1 million models** and **250k+ datasets** spanning modalities such as text, image, video, audio, and even 3D. By fostering open collaboration, Hugging Face empowers developers and researchers to innovate more quickly and ethically in AI, pushing the boundaries of what's possible.

---

## Platform Highlights

- **Models**: Explore and contribute to an extensive library of pre-trained models across various AI disciplines. Popular trending models include MiniMaxAI’s MiniMax-M2 and DeepSeek-OCR.
- **Datasets**: Access and share large-scale datasets including autonomous vehicle data from NVIDIA and language datasets like finewiki.
- **Spaces**: Launch and explore interactive AI apps and demos powered by the community, with hundreds of projects running live.
- **Community Collaboration**: Host unlimited public projects, collaborate on cutting-edge machine learning research, and build your professional ML portfolio.
- **Enterprise Solutions**: Advanced offerings for teams and enterprises include secure, scalable compute resources, access controls, and dedicated customer support.

---

## Company Culture

Hugging Face thrives as an open, ethical, and collaborative ecosystem. Its culture is deeply rooted in:

- **Open Source Mindset**: Encouraging transparency, inclusion, and the democratization of AI technologies.
- **Community Centric**: Fostering a fast-growing community that actively contributes, shares knowledge, and helps each other advance.
- **Innovation and Impact**: Supporting the next generation of AI talent to accelerate the development of responsible artificial intelligence.
- **Diversity and Learning**: Welcoming diverse voices and providing educational resources to empower users at all levels.

---

## Who Uses Hugging Face?

- **Machine Learning Engineers & Researchers**: Build and deploy models faster with access to reusable resources and collaboration tools.
- **Data Scientists & AI Enthusiasts**: Experiment with datasets and applications to prototype advanced AI solutions.
- **Developers**: Embed powerful, open-source AI capabilities into applications and products.
- **Enterprises and Teams**: Scale AI initiatives securely with enterprise-grade features and dedicated support.
- **Academics & Students**: Leverage free resources and community knowledge to learn and contribute.

---

## Careers at Hugging Face

Join a mission-driven team committed to shaping the future of AI.

Hugging Face offers opportunities for talented individuals passionate about:

- Machine Learning Engineering
- Research and Data Science
- Platform and Software Development
- Community Management and User Experience
- Enterprise Sales and Customer Success

As a member of Hugging Face, you will be part of a culture that values:

- Openness and collaboration
- Continuous learning and professional growth
- Making a global societal impact through ethical AI

---

## Get Started Today

Explore, collaborate, and innovate at the home of modern machine learning.

- Visit [huggingface.co](https://huggingface.co)
- Browse over 1 million models and 250,000 datasets
- Join the community, share your work, and build your AI portfolio
- Unlock powerful compute and enterprise solutions for your team

---

## Brand & Visual Identity

- Brand Colors: Vibrant Yellow (#FFD21E), Warm Orange (#FF9D00), Neutral Gray (#6B7280)
- Friendly and approachable visual style aligned with open collaboration and innovation

---

Hugging Face – Your partner in building an open and ethical AI future. Join the community that’s redefining what AI can do!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the pioneering AI community platform building the future of machine learning (ML). It serves as a vibrant collaboration hub where ML engineers, scientists, and enthusiasts come together to create, share, and accelerate the development of innovative AI models, datasets, and applications.

Hugging Face is best known for its extensive **open-source machine learning ecosystem** that fuels advancements across multiple data modalities, including text, image, video, audio, and even 3D.

---

## What We Offer

- **Models**: Access to over 1 million pre-trained ML models for diverse applications, updated continuously by the global community.

- **Datasets**: Browse and contribute to a repository of 250,000+ datasets supporting varied AI use cases.

- **Spaces**: Host and explore interactive AI-powered applications and demos built by the community on a collaborative platform.

- **Community**: A fast-growing, vibrant community dedicated to sharing knowledge, supporting open and ethical AI development.

- **Enterprise Solutions**: Advanced, enterprise-grade tools offering secure, scalable compute environments with access controls, designed to accelerate innovation for teams and organizations.

- **Compute Services**: Paid compute capabilities to scale your ML experiments and deployments smoothly.

---

## Why Choose Hugging Face?

- **Collaboration at Scale**: Unlimited hosting for public models, datasets, and applications encourages faster, more open innovation.

- **Open Source Foundation**: The HF Open Source Stack empowers users to build and deploy AI models efficiently.

- **Multi-Modality Support**: From text and images to audio and 3D, Hugging Face supports diverse AI research and development needs.

- **Build Your ML Portfolio**: Share your work with the world, gain recognition, and contribute to a thriving knowledge base.

- **Trending Projects & Innovation**: Stay ahead with access to the latest models like MiniMaxAI, DeepSeek-OCR, and other popular AI applications.

---

## Company Culture

Hugging Face fosters a **collaborative, inclusive, and open environment** where creativity and knowledge sharing drive progress. Their commitment to an **ethical AI future** and democratizing access to AI technologies helps empower the next generation of machine learning practitioners.

They nurture a community-first mindset that encourages continuous learning, innovation, and contribution.

---

## Customers & Community

Hugging Face's users include individual AI researchers, startups, large tech enterprises, academic institutions, and data scientists worldwide. Through their open platform, Hugging Face supports both experimental research and production-grade AI solutions.

The community-driven nature enhances innovation and diversity of ideas, empowering collective problem-solving in the AI domain.

---

## Careers & Opportunities

Hugging Face is growing rapidly and invites passionate individuals who want to shape the future of AI. Careers here offer the chance to:

- Work alongside a world-class team on cutting-edge AI tools and platforms.
- Contribute to open-source projects impacting millions worldwide.
- Thrive in a mission-driven environment promoting openness, ethical AI, and community collaboration.

Positions typically cover roles in machine learning engineering, research, software development, developer relations, and enterprise solutions.

---

## Get Started with Hugging Face

- **Explore AI Apps:** Dive into thousands of AI applications hosted on the platform.
- **Browse Models & Datasets:** Find the perfect ML assets for your projects.
- **Sign Up & Join the Community:** Build your ML portfolio, collaborate, and grow your expertise.
- **Enterprise & Compute Solutions:** Scale with advanced tools designed for team workflows.

Visit [huggingface.co](https://huggingface.co) to discover, create, and innovate with the AI community shaping tomorrow.

---

**Hugging Face**  
*The AI community building the future.*  
Empowering an open and ethical AI ecosystem for everyone.

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>